# Install Dependencies

In [ ]:
# สร้าง retriever
import chromadb
import os
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file.docs import PDFReader
from sklearn.neighbors import NearestNeighbors
from llama_index.core.schema import Document, TextNode




os.environ["OPENAI_API_KEY"] = "..." # ใส่ OpenAI API key ที่นี้
PDF_DATA_PATH="../assets/personal_data_protection_policy.pdf"
CHUNK_SIZE=1000
CHUNK_OVERLAP=250
OPENAI_TEMP=0
CHROMA_PERSISTENCE_PATH="./chroma_db"
CHROMA_COLLECTION_NAME="test4543ss332"
RETRIEVER_TOP_K=5

# Setup LLM & Embeddings

In [ ]:
embed_model = OpenAIEmbedding(model="text-embedding-3-large")

embeddings = embed_model.get_text_embedding(
    "Open AI new Embeddings models is great."
)
print(embeddings)

In [ ]:
llm = OpenAI(model="gpt-4o")

# ลองใช้งาน LLM
llm.complete("What is the capital of Thailand?")

# RAG from scratch
เราสามารถทำ RAG ได้เองโดยใช้วิธีการดังนี้
1. ตัดเอกสารออกเป็นส่วนๆ
2. เปลี่ยนแต่ละส่วนให้เป็น เวกเตอร์
3. ใช้ nearest neighbor เพื่อหาเอกสารที่ใกล้เคียงกับคำถาม
4. สร้างคำตอบจากเอกสารที่เลือกโดยใช้ LLM

In [ ]:
reader = PDFReader()
documents = reader.load_data(PDF_DATA_PATH)
embedder = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
text_splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

In [ ]:
# Preprocess documents
def ingest_documents(documents: list[Document], text_splitter: SentenceSplitter, embedder: HuggingFaceEmbedding) -> TextNode:
    pipeline = IngestionPipeline(
        transformations=[
            # 1. แบ่งเอกสารเป็น chunk ขนาด CHUNK_SIZE และมีการซ้ำกัน CHUNK_OVERLAP
            text_splitter,
            # 2. สร้างเวกเตอร์จาก OpenAI Embedding
            embedder,
        ]
    )
    nodes = pipeline.run(documents=documents)
    return nodes
    
# สร้าง retriever โดยใช้ KNN
def retrieve_documents(
    source_documents: list[TextNode],
    query: str,
    n: int = 5,
) -> list[str]:
    # embed query
    query_embedding = embedder.get_text_embedding(query)
    
    # ใช้ KNN ในการค้นหาเอกสารที่ใกล้เคียงกับ query
    search_space = [
        node.embedding for node in source_documents
    ]
    knn = NearestNeighbors(n_neighbors=n)
    knn.fit(search_space)
    
    # หาเอกสารที่ใกล้เคียงกับ query
    query_embedding = knn.kneighbors([query_embedding], return_distance=False)
        
    return [
        source_documents[idx].text for idx in query_embedding[0]
    ]
    
# ใส่ section ของเอกสารที่ search จาก retriever ลงใน prompt ของเรา 
# พร้อมกับคำถามที่ต้องการตอบ
def get_qa_prmopt(document: list[str], question: str)-> str:

    prompt = f"""
    sys: You are AI assistant your jobs is to answer the question based on the given document.
    # Document
    {document}

    # Question
    {question}
    """
    return prompt

In [ ]:
# ทดสอบ Ingest function
source_documents = ingest_documents(documents, text_splitter, embedder)
print(source_documents)

In [ ]:
# ทดสอบ Retrieve function
retrieved_documents = retrieve_documents(source_documents, "การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?")
print(retrieved_documents)

In [ ]:
# ลองดู Prompt
prompt = get_qa_prmopt(retrieved_documents, "การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?")
print(prompt)

In [ ]:
# สร้าง RAG pipeline
def answer_from_document(
    documents: list[Document],
    question: str,
    llm: OpenAI,
    max_retrieve: int = 5,
) -> str:
    # 1. แบ่งเอกสารเป็น chunk ขนาด CHUNK_SIZE และมีการซ้ำกัน CHUNK_OVERLAP ก่อนจะ embed
    ingested_documents = ingest_documents(documents, text_splitter, embedder)
    # 2. ค้นหาเอกสารที่ใกล้เคียงกับ query
    retrieved_documents = retrieve_documents(ingested_documents, question, n=max_retrieve)
    # 3. สร้าง prompt จากเอกสารที่ค้นหาได้
    prompt = get_qa_prmopt(retrieved_documents, question)
    # 4. ใช้ LLM ในการตอบคำถาม
    answer = llm.complete(prompt)
    return answer

In [ ]:
# ทดสอบการทำงานของ RAG pipeline
answer = answer_from_document(
    documents=documents,
    question="การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?",
    llm=llm,
    max_retrieve=RETRIEVER_TOP_K,
    )

In [ ]:
print(answer)

# ใช้งาน RAG โดย LlamaIndex และ Vector database

## Setup LlamaIndex Embeddings and Chunking

In [ ]:
splitter = TokenTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

Settings.node_parser = splitter

Settings.embed_model = embed_model

Settings.chunk_size=CHUNK_SIZE
Settings.chunk_overlap=CHUNK_OVERLAP

Settings.llm = llm

## อ่าน PDFs

In [ ]:
reader = PDFReader()
documents = reader.load_data(PDF_DATA_PATH)

In [ ]:
# check documents
docs = [doc.text for doc in documents]
print("Document's length: ", len(docs))
print(documents)

## Setup Chroma Vector Store and Ingest Documents

In [ ]:
def generate_retriever(documents) -> VectorStoreIndex:
    db = chromadb.Client()
    
    # [OPTIONAL] delete if existing
    try:
        db.delete_collection(CHROMA_COLLECTION_NAME)
    except ValueError:
        print("[ChromaDB] Failed to delete collection.")
        
    chroma_collection = db.create_collection(CHROMA_COLLECTION_NAME)
    
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # StorageContext is not ded like the other one
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    nodes = splitter.get_nodes_from_documents(documents)
    
    index = VectorStoreIndex(nodes, storage_context=storage_context, show_progress=True)

    return index

vector_store_index = generate_retriever(documents) 
retriever = vector_store_index.as_retriever(similarity_top_k=RETRIEVER_TOP_K)

ทดสอบการทำงานของ retriever

In [ ]:
# Sanity check for docs in the vDB
res = retriever.retrieve("การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?")

In [ ]:
print(f"Retrieve {len(res)} chunks")

# ทดสอบ our RAG workflow

In [ ]:
query_engine = vector_store_index.as_query_engine()

In [ ]:
user_query = "การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?"
response = query_engine.query(user_query)

In [ ]:
print(response)

## Using the prompt-template

ทดลองใช้  Template เพื่อให้เราได้คำตอบในรูปแบบที่เราต้องการ

In [ ]:
QUERY_PROMPT_TEMPLATE = """
# Task
From given document answer the following question.

# Instructions
- Answer the following question based on the given document.
- You need to cite the source of the answer from the document.

# Document:
{document}

# Question:
{question}

# Answer:
"""

In [ ]:
def get_answer_from_rag(
    retriever: BaseRetriever,
    llm: OpenAI,
    qa_prompt: PromptTemplate,
    question: str,
) -> str:
    # Retrieve the chunks
    chunks = retriever.retrieve(question)
    response = llm.complete(
        qa_prompt.format(
            document=chunks,
            question=question,
        )
    )
    
    return response


In [ ]:
answer = get_answer_from_rag(
    retriever=query_engine,
    llm=llm,
    qa_prompt=QUERY_PROMPT_TEMPLATE,
    question="การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?",
)

In [ ]:
print("Answer: ", answer)